In [ ]:
import os
print("--- HÜCRE 1: 128x128 Ortamı Hazırlanıyor ---")

# 1. Drive Bağlantısı
from google.colab import drive
drive.mount('/content/drive')

# 2. Kütüphaneler (Hız için Ninja şart)
print("\nKütüphaneler kuruluyor...")
!pip install click scipy requests imageio pillow psutil tqdm ninja torch torchvision opensimplex

# 3. Kod Deposu (dvschultz - En kararlısı)
if not os.path.exists("/content/stylegan2-ada-pytorch"):
    print("\nKod deposu klonlanıyor...")
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch.git

# Çalışma dizinini ayarla
os.chdir("/content/stylegan2-ada-pytorch")
print(f"\nBaşarıyla şu klasöre geçildi: {os.getcwd()}")

--- HÜCRE 1: 128x128 Ortamı Hazırlanıyor ---
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Kütüphaneler kuruluyor...

Başarıyla şu klasöre geçildi: /content/stylegan2-ada-pytorch


In [ ]:
import os
import zipfile
import io
from PIL import Image
from tqdm import tqdm

print("--- HÜCRE 2: Veri Seti 128x128'e Dönüştürülüyor ---")

# Yollar
kaynak_zip = "/content/drive/MyDrive/TUBITAK_Projesi/dataset_temiz_v1.zip"
hedef_zip = "/content/empresyonizm_128_fast.zip"
yerel_model = "/content/ffhq.pkl"
TARGET_SIZE = (128, 128)

# Transfer Modeli İndir (FFHQ)
print("Transfer modeli (FFHQ) indiriliyor...")
!wget -c https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl -O {yerel_model}

# Veri Dönüştürme
if not os.path.exists(kaynak_zip):
    print("HATA: Kaynak veri seti bulunamadı! Drive yollarını kontrol edin.")
else:
    print(f"Veri seti işleniyor: {TARGET_SIZE} piksel...")
    try:
        with zipfile.ZipFile(kaynak_zip, 'r') as zf_in:
            with zipfile.ZipFile(hedef_zip, 'w', compression=zipfile.ZIP_STORED) as zf_out:
                file_list = [f for f in zf_in.namelist() if not f.startswith('__') and f.lower().endswith(('.png', '.jpg', '.jpeg'))]

                for filename in tqdm(file_list):
                    try:
                        with zf_in.open(filename) as f:
                            img = Image.open(f).convert('RGB')
                            # Kaliteli küçültme
                            img_resized = img.resize(TARGET_SIZE, Image.LANCZOS)

                            img_byte_arr = io.BytesIO()
                            img_resized.save(img_byte_arr, format='PNG')
                            zf_out.writestr(os.path.basename(filename).split('.')[0] + ".png", img_byte_arr.getvalue())
                    except:
                        pass
        print(f"\nVeri seti hazır: {hedef_zip}")
    except Exception as e:
        print(f"Hata: {e}")

--- HÜCRE 2: Veri Seti 128x128'e Dönüştürülüyor ---
Transfer modeli (FFHQ) indiriliyor...
--2025-12-05 17:19:33--  https://nvlabs-fi-cdn.nvidia.com/stylegan2-ada-pytorch/pretrained/ffhq.pkl
Resolving nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)... 99.84.215.15, 99.84.215.98, 99.84.215.108, ...
Connecting to nvlabs-fi-cdn.nvidia.com (nvlabs-fi-cdn.nvidia.com)|99.84.215.15|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

Veri seti işleniyor: (128, 128) piksel...


100%|██████████| 683/683 [00:45<00:00, 14.95it/s]


Veri seti hazır: /content/empresyonizm_128_fast.zip


In [ ]:
import os

print("--- HÜCRE 3: 128x128 Eğitim SIFIRDAN Başlatılıyor (EN GARANTİ YOL) ---")

# Ayarlar
OUTDIR = "/content/drive/MyDrive/TUBITAK_Projesi/results_128_FINAL_V2"
DATA = "/content/empresyonizm_128_fast.zip"

# NOT: 'resume' parametresini kaldırdım. Sıfırdan başlayacak.
# Bu sayede 'boyut uyuşmazlığı' hatası ihtimali %0 olur.

GPUS = 1
KIMG = 2500
GAMMA = 10       # Sıfırdan eğitim için stabilite sağlar
AUGPIPE = "bgc"
METRICS = "none"

# --- GÜVENLİ AYARLAR ---
BATCH_SIZE = 32  # 128px için hızlı
WORKERS = 1      # RAM dostu (Çökmez)
SNAP = 10        # Her 10 kimg'de bir (15-20 dk'da bir) resim üretir
# -----------------------

print(f"Veri: {DATA}")
print(f"Çıktı: {OUTDIR}")
print("Mod: Sıfırdan Eğitim (Hatasız)")

!python train.py --outdir={OUTDIR} \
                 --data={DATA} \
                 --gpus={GPUS} \
                 --kimg={KIMG} \
                 --gamma={GAMMA} \
                 --batch={BATCH_SIZE} \
                 --augpipe={AUGPIPE} \
                 --metrics={METRICS} \
                 --workers={WORKERS} \
                 --snap={SNAP}

Streaming output truncated to the last 5000 lines.
ModuleNotFoundError: No module named 'upfirdn2d_plugin'

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback.format_exc())
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
/content/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/content/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/stylegan2-ada-pytorch/torch_utils/custom_ops.py", line 111, in get_plugin
    module = importlib.import_module(module_name)
             ^^^^^^^^^

In [ ]:
import os
print("--- HÜCRE 1: Ortam Hazırlanıyor ---")

# 1. Drive Bağlantısı
from google.colab import drive
drive.mount('/content/drive')

# 2. Gerekli Kütüphaneler
print("\nKütüphaneler kuruluyor...")
!pip install click scipy requests imageio pillow psutil tqdm ninja torch torchvision opensimplex

# 3. Kod Deposu
if not os.path.exists("/content/stylegan2-ada-pytorch"):
    print("\nKod deposu klonlanıyor...")
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch.git

# Çalışma dizinini ayarla
os.chdir("/content/stylegan2-ada-pytorch")
print(f"\nBaşarıyla şu klasöre geçildi: {os.getcwd()}")

--- HÜCRE 1: Ortam Hazırlanıyor ---
Mounted at /content/drive

Kütüphaneler kuruluyor...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.0/268.0 kB 11.5 MB/s eta 0:00:00

Kod deposu klonlanıyor...
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 577, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 577 (delta 16), reused 12 (delta 12), pack-reused 552 (from 2)
Receiving objects: 100% (577/577), 8.43 MiB | 31.16 MiB/s, done.
Resolving deltas: 100% (329/329), done.

Başarıyla şu klasöre geçildi: /content/stylegan2-ada-pytorch


In [ ]:
import os
import glob
import shutil
import zipfile
import io
from PIL import Image
from tqdm import tqdm

print("--- HÜCRE 2: Dosyalar ve Model Hazırlanıyor ---")

# 1. EN SON MODELİ BUL (Otomatik)
# -------------------------------
print("\n1. En son kaydedilen model aranıyor...")
# Drive'da 128'lik sonuçların olduğu klasörleri tara
search_path = "/content/drive/MyDrive/TUBITAK_Projesi/results_128*/**/*.pkl"
pkl_files = glob.glob(search_path, recursive=True)

if len(pkl_files) > 0:
    # Dosyaları ismine göre sırala (000840 en sonda olacak)
    pkl_files.sort()
    latest_pkl = pkl_files[-1] # En sondaki dosyayı al

    print(f"BULUNDU: {os.path.basename(latest_pkl)}")
    print(f"Tam Yol: {latest_pkl}")

    # Modeli 'resume.pkl' olarak kaydet (Kolaylık olsun diye)
    resume_path = "/content/resume.pkl"
    shutil.copy(latest_pkl, resume_path)
    print("Model devam etmek için hazırlandı.")
else:
    print("HATA: Hiçbir .pkl dosyası bulunamadı! Lütfen Drive'ı kontrol edin.")
    # Acil durum: Eğer bulamazsa, ffhq'dan başlatmak zorunda kalırız ama bulacaktır.

# 2. VERİ SETİNİ HAZIRLA
# -------------------------------
print("\n2. Veri seti 128x128 olarak hazırlanıyor...")
kaynak_zip = "/content/drive/MyDrive/TUBITAK_Projesi/dataset_temiz_v1.zip"
hedef_zip = "/content/empresyonizm_128_fast.zip"
TARGET_SIZE = (128, 128)

if not os.path.exists(hedef_zip): # Eğer zaten yoksa oluştur
    if os.path.exists(kaynak_zip):
        try:
            with zipfile.ZipFile(kaynak_zip, 'r') as zf_in:
                with zipfile.ZipFile(hedef_zip, 'w', compression=zipfile.ZIP_STORED) as zf_out:
                    file_list = [f for f in zf_in.namelist() if not f.startswith('__') and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                    for filename in tqdm(file_list):
                        with zf_in.open(filename) as f:
                            img = Image.open(f).convert('RGB')
                            img_resized = img.resize(TARGET_SIZE, Image.LANCZOS)
                            img_byte_arr = io.BytesIO()
                            img_resized.save(img_byte_arr, format='PNG')
                            zf_out.writestr(os.path.basename(filename).split('.')[0] + ".png", img_byte_arr.getvalue())
            print("Veri seti hazır.")
        except Exception as e:
            print(f"Veri hatası: {e}")
    else:
        print("Kaynak veri seti (dataset_temiz_v1.zip) bulunamadı!")
else:
    print("Veri seti zaten hazır (Hızlı geçiş).")

--- HÜCRE 2: Dosyalar ve Model Hazırlanıyor ---

1. En son kaydedilen model aranıyor...
BULUNDU: network-snapshot-000840.pkl
Tam Yol: /content/drive/MyDrive/TUBITAK_Projesi/results_128_FINAL_V2/00000-empresyonizm_128_fast-auto1-gamma10-kimg2500-batch32-bgc/network-snapshot-000840.pkl
Model devam etmek için hazırlandı.

2. Veri seti 128x128 olarak hazırlanıyor...
Veri seti zaten hazır (Hızlı geçiş).


In [ ]:
import os

print("--- HÜCRE 3: Eğitim Kaldığı Yerden (840 -> 2500) Devam Ediyor ---")

# Ayarlar
OUTDIR = "/content/drive/MyDrive/TUBITAK_Projesi/results_128_RESUME_FINAL"
DATA = "/content/empresyonizm_128_fast.zip"
RESUME = "/content/resume.pkl" # Hücre 2'de hazırladığımız dosya

GPUS = 1
KIMG = 2500      # Hedef
GAMMA = 1
AUGPIPE = "bgc"
METRICS = "none"

# --- HIZ AYARLARI ---
BATCH_SIZE = 32
WORKERS = 1      # RAM korumalı (Çökmemesi için 1 yaptık)
SNAP = 10        # Her 10 kimg'de bir (Sık sık) kayıt al
# --------------------

print(f"Başlangıç Modeli: {RESUME}")
print(f"Hedef: {KIMG} kimg")

!python train.py --outdir={OUTDIR} \
                 --data={DATA} \
                 --resume={RESUME} \
                 --gpus={GPUS} \
                 --kimg={KIMG} \
                 --gamma={GAMMA} \
                 --batch={BATCH_SIZE} \
                 --augpipe={AUGPIPE} \
                 --metrics={METRICS} \
                 --workers={WORKERS} \
                 --snap={SNAP}

Streaming output truncated to the last 5000 lines.
ModuleNotFoundError: No module named 'upfirdn2d_plugin'

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback.format_exc())
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
/content/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/content/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/stylegan2-ada-pytorch/torch_utils/custom_ops.py", line 111, in get_plugin
    module = importlib.import_module(module_name)
             ^^^^^^^^^

In [ ]:
import os
print("HÜCRE 1: Ortam Hazırlanıyor")

# 1. Drive Bağlantısı
from google.colab import drive
drive.mount('/content/drive')

# 2. Gerekli Kütüphaneler
print("\nKütüphaneler kuruluyor...")
!pip install click scipy requests imageio pillow psutil tqdm ninja torch torchvision opensimplex

# 3. Kod Deposu
if not os.path.exists("/content/stylegan2-ada-pytorch"):
    print("\nKod deposu klonlanıyor...")
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch.git

# Çalışma dizinini ayarla
os.chdir("/content/stylegan2-ada-pytorch")
print(f"\nBaşarıyla şu klasöre geçildi: {os.getcwd()}")

HÜCRE 1: Ortam Hazırlanıyor
Mounted at /content/drive

Kütüphaneler kuruluyor...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.0/268.0 kB 19.1 MB/s eta 0:00:00

Kod deposu klonlanıyor...
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 577, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 577 (delta 21), reused 17 (delta 17), pack-reused 546 (from 2)
Receiving objects: 100% (577/577), 8.43 MiB | 27.06 MiB/s, done.
Resolving deltas: 100% (329/329), done.

Başarıyla şu klasöre geçildi: /content/stylegan2-ada-pytorch


In [ ]:
import os
import glob
import shutil
import zipfile
import io
from PIL import Image
from tqdm import tqdm

print("--- HÜCRE 2: Dosyalar Hazırlanıyor ---")

# 1. EN GÜNCEL MODELİ BUL (Akıllı Arama)

print("\n1. En son eğitilmiş model aranıyor...")
# Tüm 128'lik klasörleri tara
search_path = "/content/drive/MyDrive/TUBITAK_Projesi/results_128*/**/*.pkl"
all_pkls = glob.glob(search_path, recursive=True)

if len(all_pkls) > 0:
    # Dosyaları değiştirilme tarihine göre sırala (En yeni en sonda)
    all_pkls.sort(key=os.path.getmtime)
    latest_pkl = all_pkls[-1] # En son oluşturulan dosya

    print(f"BULUNDU: {os.path.basename(latest_pkl)}")
    print(f"Klasör: {os.path.dirname(latest_pkl)}")

    # Yerel diske kopyala
    resume_path = "/content/resume_final.pkl"
    shutil.copy(latest_pkl, resume_path)
    print("Model devam etmek için hazırlandı (/content/resume_final.pkl).")
else:
    print("HATA: Hiçbir .pkl dosyası bulunamadı! Lütfen Drive'ı kontrol edin.")
    import sys; sys.exit()

# 2. VERİ SETİNİ HAZIRLA

print("\n2. Veri seti 128x128 olarak hazırlanıyor...")
kaynak_zip = "/content/drive/MyDrive/TUBITAK_Projesi/dataset_temiz_v1.zip"
hedef_zip = "/content/empresyonizm_128_ready.zip"
TARGET_SIZE = (128, 128)

if not os.path.exists(hedef_zip):
    if os.path.exists(kaynak_zip):
        try:
            with zipfile.ZipFile(kaynak_zip, 'r') as zf_in:
                with zipfile.ZipFile(hedef_zip, 'w', compression=zipfile.ZIP_STORED) as zf_out:
                    file_list = [f for f in zf_in.namelist() if not f.startswith('__') and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
                    for filename in tqdm(file_list, desc="Dönüştürülüyor"):
                        with zf_in.open(filename) as f:
                            img = Image.open(f).convert('RGB')
                            img_resized = img.resize(TARGET_SIZE, Image.LANCZOS)
                            img_byte_arr = io.BytesIO()
                            img_resized.save(img_byte_arr, format='PNG')
                            zf_out.writestr(os.path.basename(filename).split('.')[0] + ".png", img_byte_arr.getvalue())
            print("Veri seti hazır.")
        except Exception as e:
            print(f"Veri hatası: {e}")
    else:
        print("Kaynak veri seti bulunamadı!")
else:
    print("Veri seti zaten hazır.")

--- HÜCRE 2: Dosyalar Hazırlanıyor ---

1. En son eğitilmiş model aranıyor...
BULUNDU: network-snapshot-000720.pkl
Klasör: /content/drive/MyDrive/TUBITAK_Projesi/results_128_RESUME_FINAL/00000-empresyonizm_128_fast-auto1-gamma1-kimg2500-batch32-bgc-resumecustom
Model devam etmek için hazırlandı (/content/resume_final.pkl).

2. Veri seti 128x128 olarak hazırlanıyor...


Dönüştürülüyor:  71%|███████   | 482/683 [00:33<00:17, 11.51it/s]/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (171668516 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
Dönüştürülüyor: 100%|██████████| 683/683 [00:49<00:00, 13.78it/s]

Veri seti hazır.


In [ ]:
import os

print("--- HÜCRE 3: 128x128 Eğitim FİNAL TURU Başlatılıyor ---")

# Ayarlar
OUTDIR = "/content/drive/MyDrive/TUBITAK_Projesi/results_128_FINAL_PART3"
DATA = "/content/empresyonizm_128_ready.zip"
RESUME = "/content/resume_final.pkl"

GPUS = 1
KIMG = 2500      # Hedef
GAMMA = 1
AUGPIPE = "bgc"
METRICS = "none"

# --- HIZ AYARLARI ---
BATCH_SIZE = 32
WORKERS = 1      # RAM dostu
SNAP = 10        # Her 10 kimg'de bir kayıt al


print(f"Başlangıç Modeli: {RESUME}")
print(f"Hedef: {KIMG} kimg")

!python train.py --outdir={OUTDIR} \
                 --data={DATA} \
                 --resume={RESUME} \
                 --gpus={GPUS} \
                 --kimg={KIMG} \
                 --gamma={GAMMA} \
                 --batch={BATCH_SIZE} \
                 --augpipe={AUGPIPE} \
                 --metrics={METRICS} \
                 --workers={WORKERS} \
                 --snap={SNAP}

Görüntülenen çıkış son 5000 satıra kısaltıldı.
ModuleNotFoundError: No module named 'upfirdn2d_plugin'

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback.format_exc())
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
/content/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "/content/stylegan2-ada-pytorch/torch_utils/ops/upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/stylegan2-ada-pytorch/torch_utils/custom_ops.py", line 111, in get_plugin
    module = importlib.import_module(module_name)
             ^^^^^^^^^^^^^